In [ ]:
import pandas as pd

In [ ]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.1 MB/s eta 0:00:00


In [ ]:
import praw

In [ ]:
reddit = praw.Reddit(client_id = "eD7UbErHWUuBuDqdI9UxwA",client_secret = "ngG6iJ153Q9FqDezGf2Ztda3JjUyxg",redirect_url = "http://localhost:8080",
                     user_agent = "siddharthsesh")

In [ ]:
posts = reddit.subreddit('MachineLearning+artificial+datascience').top(time_filter = "all",limit = 1000)

posts_df = []

for post in posts:
  posts_df.append({'post_id':post.id,
                   'subreddit':post.subreddit,
                   'post_title':post.title,
                   })
posts_dfa = pd.DataFrame(posts_df)
posts_dfa.to_csv('DS_ML_AI_posts.csv',header = True, index = False)

In [ ]:
posts_dfa = pd.read_csv('DS_ML_AI_posts.csv')

In [ ]:
comments_list = []

for post_id in posts_dfa['post_id']:
  submission = reddit.submission(post_id)
  submission.comments.replace_more(limit = None)
  for comment in submission.comments.list():
    comments_list.append({'post_id':post_id,'comment':comment.body})
comment_df = pd.DataFrame(comments_list)
comment_df.to_csv('DS_ML_AI_comments.csv',header = True,index = False)

In [ ]:
comment_df = pd.read_csv('DS_ML_AI_comments.csv')

In [ ]:
comment_df.head()

,post_id,comment
0,gh1dj9,Twitter thread: [https://twitter.com/cyrildiag...
1,gh1dj9,Simple yet very useful. Thank you for sharing ...
2,gh1dj9,The future 🤯
3,gh1dj9,Ohh the nightmare of making this into a stable...
4,gh1dj9,"Almost guaranteed, Apple will copy your idea i..."


In [ ]:
posts_df = posts_dfa[['post_id','post_title']]

In [ ]:
comments_posts_df = posts_df.merge(comment_df, on='post_id', how='left')

In [ ]:
comments_posts_df = comments_posts_df.astype('str')

In [ ]:
agg_comments = comments_posts_df.groupby('post_title')['comment'].apply('. '.join).reset_index()

In [ ]:
agg_comments['combined_text'] = agg_comments.astype(str).agg('. '.join, axis=1)
all_text = ' '.join(agg_comments['combined_text'])

In [ ]:
f = open("text/all_text_reddit.txt", "w")
f.write(all_text)
f.close()

In [ ]:
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

In [ ]:
!pip install langchain

In [ ]:
!pip install streamlit

In [ ]:
from huggingface_hub import login
login()

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers
!pip -q install langchain
!pip -q install gradio

In [ ]:
!pip -q install peft chromadb
!pip -q install unstructured
!pip install -q sentence_transformers
!pip -q install pypdf

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import MarkdownHeaderTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("/content/text/all_text_reddit.txt")

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 20,
    length_function = len,
)

In [ ]:
pages = loader.load_and_split(text_splitter)

In [ ]:
db = Chroma.from_documents(pages, HuggingFaceEmbeddings(), persist_directory = '/content/db')

In [ ]:
instruction = "Given the context that has been provided. \n {context}, Answer the following question - \n{question}"

system_prompt = """You are an expert in chess.
You will be given a context to answer from. Be precise in your answers wherever possible.
In case you are sure you don't know the answer then you say that based on the context you don't know the answer.
In all other instances you provide an answer to the best of your capability. Cite urls when you can access them related to the context."""

get_prompt(instruction, system_prompt)

In [ ]:
!pip install chromadb

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [ ]:
bnb_config = BitsAndBytesConfig(load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False)

In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

In [ ]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

In [ ]:
memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5,
    return_messages=True
)

In [ ]:
retriever = db.as_retriever()

In [ ]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [ ]:
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 2}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [ ]:
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
        model = "TheBloke/Llama-2-7B-Chat-GGML",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5
    )
    return llm

In [ ]:
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

In [ ]:
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [ ]:
import chainlit as cl
@cl.on_chat_start
async def start():
    chain = qa_bot()
    msg = cl.Message(content="Starting the bot...")
    await msg.send()
    msg.content = "Hi, Welcome to Medical Bot. What is your query?"
    await msg.update()

    cl.user_session.set("chain", chain)

In [ ]:
@cl.on_message
async def main(message: cl.Message):
    chain = cl.user_session.get("chain")
    cb = cl.AsyncLangchainCallbackHandler(
        stream_final_answer=True, answer_prefix_tokens=["FINAL", "ANSWER"]
    )
    cb.answer_reached = True
    res = await chain.acall(message.content, callbacks=[cb])
    answer = res["result"]